## datagen.py

In [1]:
from __future__ import print_function
import os
import io
import sys
import random
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image

In [2]:
class ListDataset(data.Dataset):
    def __init__(self, root, transform):
        self.root = root
        self.transform = transform
        self.fname = []
        self.age = []
        self.gender = []
        
        for filename in os.listdir(root):
            self.fname.append(filename)
            filename_split = filename.split("_")
            self.age.append(int(filename_split[0]))
            self.gender.append(int(filename_split[1]))
            
        self.num_imgs = len(self.fname)
    
    def __getitem__(self, idx):
        """Load image."""
        fname = self.fname[idx]
        age = self.age[idx]
        gender = self.gender[idx]
        
        img = Image.open(os.path.join(self.root, fname))
        img = self.transform(img)
        return img, age, gender
    
    def __len__(self):
        return self.num_imgs 

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

from cnn_finetune import make_model

from net import AGNet
from loss import AGLoss

from torch.autograd import Variable

In [4]:
batch_size = 32
test_batch_size = 64
epochs = 50
learning_rate = 0.01
momentum = 0.9
dropout = 0.2
start_epoch = 0

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [6]:
list_classes = []

In [7]:
for i in range(1, 117):
    list_classes.append(str(i))

In [8]:
classes = tuple(list_classes)

In [9]:
# model = make_model(
#     'resnet18',
#     pretrained=True,
#     num_classes=len(classes),
#     dropout_p=dropout,
# )

In [10]:
transform_train = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.RandomCrop(150, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)
    )
])

In [11]:
trainset = ListDataset(root='../data/UTKFace/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=8)

In [12]:
transform_test = transforms.Compose([
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225))
])
testset = ListDataset(root='../data/test/', transform=transform_test)

In [13]:
net = AGNet()
# net.load_state_dict(torch.load('/home/neosai/.torch/models/resnet18-5c106cde.pth'))
# net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
net.cuda()
criterion = AGLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=1e-4)

In [14]:
def accuracy(age_preds, age_targets, gender_preds, gender_targets):
    AGE_TOLERANCE = 3
    age_prob = F.softmax(age_preds)
    age_expect = torch.sum(Variable(torch.arange(1, 117)).cuda() * age_prob, 1)
    
    age_correct = ((age_expect - age_targets.float()).abs() < AGE_TOLERANCE).long().sum().cpu().data[0]
    
    gender_preds = F.sigmoid(gender_preds)
    gender_preds = (gender_preds > 0.5).long()
    gender_correct = (gender_preds == gender_targets.long()).long().cpu().sum().data[0]
    return age_correct, gender_correct

In [15]:
def train(epoch):
    net.train()
    train_loss = 0
    total = 0
    age_correct = 0
    gender_correct = 0
    for batch_idx, (inputs, age_targets, gender_targets) in enumerate(trainloader):
        inputs = Variable(inputs.cuda())
        age_targets = Variable(age_targets.cuda())
        gender_targets = Variable(gender_targets.cuda())
        optimizer.zero_grad()
        age_preds, gender_preds = net(inputs)
        loss = criterion(age_preds, age_targets, gender_preds, gender_targets).long()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data[0]
        age_correct_i, gender_correct_i = accuracy(age_preds,
                                                  age_targets, gender_preds,
                                                  gender_targets)
        age_correct += age_correct_i
        gender_correct += gender_correct_i
        total += len(inputs)
        
        print('train_loss: %.3f | avg_loss: %.3f | age_prec: %.3f (%d/%d) | gender_prec: %.3f (%d/%d)  [%d/%d]'  \
            % (loss.data[0], train_loss/(batch_idx+1),      \
               100.*age_correct/total, age_correct, total,  \
               100.*gender_correct/total, gender_correct, total,    \
               batch_idx+1, len(trainloader)))

In [16]:
# Test
def test(epoch):
    print('\nTest')
    net.eval()
    test_loss = 0
    total = 0
    age_correct = 0
    gender_correct = 0
    for batch_idx, (inputs, age_targets, gender_targets) in enumerate(testloader):
        inputs = Variable(inputs.cuda(), volatile=True)
        age_targets = Variable(age_targets.cuda())
        gender_targets = Variable(gender_targets.cuda())

        age_preds, gender_preds = net(inputs)
        loss = criterion(age_preds, age_targets, gender_preds, gender_targets)

        test_loss += loss.data[0]
        age_correct_i, gender_correct_i = accuracy(
            age_preds, age_targets, gender_preds, gender_targets)
        age_correct += age_correct_i
        gender_correct += gender_correct_i
        total += len(inputs)
        print('test_loss: %.3f | avg_loss: %.3f | age_prec: %.3f (%d/%d) | gender_prec: %.3f (%d/%d)  [%d/%d]' \
            % (loss.data[0], test_loss/(batch_idx+1),      \
               100.*age_correct/total, age_correct, total,  \
               100.*gender_correct/total, gender_correct, total, \
               batch_idx+1, len(trainloader)))

    # Save checkpoint
    global best_correct
    if age_correct + gender_correct > best_correct:
        print('Saving..')
        best_correct = age_correct + gender_correct
        state = {
            'net': net.module.state_dict(),
            'correct': best_correct,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')

In [17]:
for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch)

/home/neosai/Documents/projects/predict_age_gender/src/loss.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  age_prob = F.softmax(age_preds).long().cpu()


<class 'torch.Tensor'>


RuntimeError: smooth_l1_loss_forward is not implemented for type torch.cuda.LongTensor